In [1]:
import os
import os.path as osp
import argparse
from datetime import date
import json
import random
import time
from pathlib import Path
import numpy as np
import numpy.linalg as LA
import torch.linalg
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2
import csv

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import util.misc as utils
from config import cfg

import pandas as pd
import glm

/home/kmuvcl/anaconda3/envs/cuti/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "/home/kmuvcl/source/oldCuTi/CuTi/matterport/mp3d_planercnn_json/cached_set_test.json"
root = "/home/kmuvcl/source/oldCuTi/CuTi/matterport/"

In [3]:
with open(path) as f:
        split = json.load(f)

dset = split

In [4]:
def _sqrt_positive_part(x: torch.Tensor) -> torch.Tensor:
    """
    Returns torch.sqrt(torch.max(0, x))
    but with a zero subgradient where x is 0.
    """
    ret = torch.zeros_like(x)
    positive_mask = x > 0
    ret[positive_mask] = torch.sqrt(x[positive_mask])
    return ret


def matrix_to_quaternion(matrix: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as rotation matrices to quaternions.

    Args:
        matrix: Rotation matrices as tensor of shape (..., 3, 3).

    Returns:
        quaternions with real part first, as tensor of shape (..., 4).
    """
    if matrix.size(-1) != 3 or matrix.size(-2) != 3:
        raise ValueError(f"Invalid rotation matrix shape {matrix.shape}.")

    batch_dim = matrix.shape[:-2]
    m00, m01, m02, m10, m11, m12, m20, m21, m22 = torch.unbind(
        matrix.reshape(batch_dim + (9,)), dim=-1
    )

    q_abs = _sqrt_positive_part(
        torch.stack(
            [
                1.0 + m00 + m11 + m22,
                1.0 + m00 - m11 - m22,
                1.0 - m00 + m11 - m22,
                1.0 - m00 - m11 + m22,
            ],
            dim=-1,
        )
    )

    # we produce the desired quaternion multiplied by each of r, i, j, k
    quat_by_rijk = torch.stack(
        [
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([q_abs[..., 0] ** 2, m21 - m12, m02 - m20, m10 - m01], dim=-1),
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([m21 - m12, q_abs[..., 1] ** 2, m10 + m01, m02 + m20], dim=-1),
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([m02 - m20, m10 + m01, q_abs[..., 2] ** 2, m12 + m21], dim=-1),
            # pyre-fixme[58]: `**` is not supported for operand types `Tensor` and
            #  `int`.
            torch.stack([m10 - m01, m20 + m02, m21 + m12, q_abs[..., 3] ** 2], dim=-1),
        ],
        dim=-2,
    )

    # We floor here at 0.1 but the exact level is not important; if q_abs is small,
    # the candidate won't be picked.
    flr = torch.tensor(0.1).to(dtype=q_abs.dtype, device=q_abs.device)
    quat_candidates = quat_by_rijk / (2.0 * q_abs[..., None].max(flr))

    # if not for numerical problems, quat_candidates[i] should be same (up to a sign),
    # forall i; we pick the best-conditioned one (with the largest denominator)

    return quat_candidates[
        F.one_hot(q_abs.argmax(dim=-1), num_classes=4) > 0.5, :
    ].reshape(batch_dim + (4,))

def quaternion_to_matrix(quaternions: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as quaternions to rotation matrices.

    Args:
        quaternions: quaternions with real part first,
            as tensor of shape (..., 4).

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """
    r, i, j, k = torch.unbind(quaternions, -1)
    # pyre-fixme[58]: `/` is not supported for operand types `float` and `Tensor`.
    two_s = 2.0 / (quaternions * quaternions).sum(-1)

    o = torch.stack(
        (
            1 - two_s * (j * j + k * k),
            two_s * (i * j - k * r),
            two_s * (i * k + j * r),
            two_s * (i * j + k * r),
            1 - two_s * (i * i + k * k),
            two_s * (j * k - i * r),
            two_s * (i * k - j * r),
            two_s * (j * k + i * r),
            1 - two_s * (i * i + j * j),
        ),
        -1,
    )
    return o.reshape(quaternions.shape[:-1] + (3, 3))

In [5]:
len(dset['data'])

7996

In [6]:
df1 = pd.DataFrame(index=range(0,len(dset['data'])),columns={'filename':[],'line_file_name':[],'vp0x':[],'vp0y':[],'vp0z':[],'vp1x':[],'vp1y':[],'vp1z':[],'vp2x':[],'vp2y':[],'vp2z':[]})
df2 = pd.DataFrame(index=range(0,len(dset['data'])),columns={'filename':[],'line_file_name':[],'vp0x':[],'vp0y':[],'vp0z':[],'vp1x':[],'vp1y':[],'vp1z':[],'vp2x':[],'vp2y':[],'vp2z':[]})

In [7]:
df1

,filename,line_file_name,vp0x,vp0y,vp0z,vp1x,vp1y,vp1z,vp2x,vp2y,vp2z
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
in_matrix = torch.tensor([
    [1,0,0],
    [0,1,0],
    [320,240,1]
],dtype=torch.float)

In [30]:
in_matrix

tensor([[  1.,   0.,   0.],
        [  0.,   1.,   0.],
        [320., 240.,   1.]])

In [31]:
dset['data'][0]['rel_pose']['rotation']

[0.9207172033825632, 0.0, -0.38306085185081906, -0.07445948680131997]

In [ ]:
import torch.linalg
import numpy as np
# for i in range(2):
for i in tqdm(range(len(dset['data']))):
    images = []
    lines = []
    datapath = ""
    focal_length = 517.97/240
    for imgnum in ['0', '1']:
        
        img_name = os.path.join(datapath, '/'.join(str(dset['data'][i][imgnum]['file_name']).split('/')[6:]))
        images.append(img_name)
        
        line_name = img_name.split("/")
        line_name[2] = img_name.split("/")[2].split(".")[0] + "_line.csv"
        line_name = "/".join(line_name)
        lines.append(line_name)
        #print(line_name)
    df1['filename'][i] = images[0]
    df2['filename'][i] = images[1]
    df1['line_file_name'][i]=lines[0]
    df2['line_file_name'][i]=lines[1]
    
    R1_Q = dset['data'][i]['0']['camera']['rotation']
    R2_Q = dset['data'][i]['1']['camera']['rotation']
    R1 = quaternion_to_matrix(torch.tensor(R1_Q))
    R2 = quaternion_to_matrix(torch.tensor(R2_Q))
    
    R1 = R1 #@ in_matrix
    R2 = R2 #@ in_matrix
    
    R1[0] = R1[0]/R1[0][2]
    R1[1] = R1[1]/R1[1][2]
    R1[2] = R1[2]/R1[2][2]
    
    R2[0] = R2[0]/R2[0][2]
    R2[1] = R2[1]/R2[1][2]
    R2[2] = R2[2]/R2[2][2]
    
    # R1_img = np.array([
    #     [R1[0][0],R1[0][1],R1[0][2]],
    #     [R1[1][0],R1[1][1],R1[1][2]],
    #     [R1[2][0],R1[2][1],R1[2][2]]
    #     ])
    # R2_img = np.array([
    #     [R2[0][0],R2[0][1],R2[0][2]],
    #     [R2[1][0],R2[1][1],R2[1][2]],
    #     [R2[2][0],R2[2][1],R2[][2]]
    #     ])
    cross1 = torch.cross(R1[0],R1[1])
    dot1 = F.cosine_similarity(cross1,R1[2], dim=-1)
    #dot1 = torch.dot(cross1,R1[2])
    cross2 = torch.cross(R2[0],R2[1])
    dot2 = F.cosine_similarity(cross2,R2[2], dim=-1)
    #dot2 = torch.dot(cross2,R2[2])
    if(dot1<0 or dot2<0):
        print(dot1,dot2)
    # norm_x = R1[0]/R1[0][2]
    # print("R1[0]/R1[0][2]",norm_x)
    # nor = F.normalize(norm_x, p=2, dim=-1)
    # print("nor",nor)
    # print("R1[0]",R1[0])
        
    # cross1 = torch.cross(R1[0]/R1[0][2],R1[1]/R1[1][2])
    # dot1 = torch.dot(cross1,R1[2]/R1[2][2])
    
    # cross2 = torch.cross(R2[0]/R2[0][2],R2[1]/R2[1][2])
    # dot2 = torch.dot(cross2,R2[2]/R2[2][2])
    
    # if(dot1<0 or dot2<0):
    #     print(dot1,dot2)
    
    # print("R1",R1)
    # print("R1[0][0]",R1[0][0])
    # print("R1[0][1]",R1[0][1])
    # print("R1[0][2]",R1[0][2])
        

    
    df1['vp0x'][i]=(R1[0][0]).tolist()
    df1['vp0y'][i]=(R1[0][1]).tolist()
    df1['vp0z'][i]=(R1[0][2]).tolist()
    df1['vp1x'][i]=(R1[1][0]).tolist()
    df1['vp1y'][i]=(R1[1][1]).tolist()
    df1['vp1z'][i]= (R1[1][2]).tolist()
    df1['vp2x'][i]=(R1[2][0]).tolist()
    df1['vp2y'][i]=(R1[2][1]).tolist()
    df1['vp2z'][i]= (R1[2][2]).tolist()

    df2['vp0x'][i]= (R2[0][0]).tolist()
    df2['vp0y'][i]= (R2[0][1]).tolist()
    df2['vp0z'][i]= (R2[0][2]).tolist()
    df2['vp1x'][i]= (R2[1][0]).tolist()
    df2['vp1y'][i]= (R2[1][1]).tolist()
    df2['vp1z'][i]= (R2[1][2]).tolist()
    df2['vp2x'][i]= (R2[2][0]).tolist()
    df2['vp2y'][i]= (R2[2][1]).tolist()
    df2['vp2z'][i]= (R2[2][2]).tolist()

GT rel_pose 저장하기

In [8]:
df = pd.DataFrame(index=range(0,len(dset['data'])),columns={'filename1':[],'filename2':[],'line_file_name1':[],'line_file_name2':[], 'rel_rot':[],'rel_tran':[]})

In [18]:
import torch.linalg
import numpy as np
from torch import nn
RQ_loss = []
for i in tqdm(range(len(dset['data']))):
    images = []
    lines = []
    datapath = ""
    focal_length = 517.97/240
    for imgnum in ['0', '1']:
        
        img_name = os.path.join(datapath, '/'.join(str(dset['data'][i][imgnum]['file_name']).split('/')[6:]))
        images.append(img_name)
        
        line_name = img_name.split("/")
        line_name[2] = img_name.split("/")[2].split(".")[0] + "_line.csv"
        line_name = "/".join(line_name)
        lines.append(line_name)
        #print(line_name)
    # df1['filename'][i] = images[0]
    # df2['filename'][i] = images[1]
    # df1['line_file_name'][i]=lines[0]
    # df2['line_file_name'][i]=lines[1]
    GT_Q = torch.tensor(dset['data'][i]['rel_pose']['rotation'])
    if GT_Q[0]<0:
        GT_Q = -GT_Q
    
    R1_Q = dset['data'][i]['0']['camera']['rotation']
    R2_Q = dset['data'][i]['1']['camera']['rotation']
    R1 = quaternion_to_matrix(torch.tensor(R1_Q))
    R2 = quaternion_to_matrix(torch.tensor(R2_Q))
    
    R1 = R1 #@ in_matrix
    R2 = R2 #@ in_matrix
    
    # R1[0] = R1[0]/R1[0][2]
    # R1[1] = R1[1]/R1[1][2]
    # R1[2] = R1[2]/R1[2][2]
    
    # R2[0] = R2[0]/R2[0][2]
    # R2[1] = R2[1]/R2[1][2]
    # R2[2] = R2[2]/R2[2][2]
    
    R = R2.T@R1
    Q = matrix_to_quaternion(R)
    if Q[0]<0:
        Q = -Q
    l2_loss = nn.MSELoss()

    RQ_loss.append(l2_loss(Q,GT_Q))
    print(l2_loss(Q,GT_Q))
    
    
    


  5%|▍         | 392/7996 [00:00<00:03, 1959.86it/s]

tensor(2.2230e-16)
tensor(2.7851e-16)
tensor(3.0307e-16)
tensor(1.1783e-15)
tensor(3.2412e-16)
tensor(5.3821e-18)
tensor(1.8039e-17)
tensor(2.3176e-16)
tensor(2.2985e-16)
tensor(1.1703e-15)
tensor(3.5539e-15)
tensor(2.8909e-15)
tensor(4.4422e-15)
tensor(2.3390e-16)
tensor(9.4369e-16)
tensor(4.4964e-15)
tensor(9.1038e-15)
tensor(1.1660e-15)
tensor(2.3392e-17)
tensor(1.8381e-15)
tensor(2.2409e-16)
tensor(1.7802e-15)
tensor(1.4117e-15)
tensor(2.5922e-17)
tensor(3.4289e-16)
tensor(9.5213e-16)
tensor(1.5054e-17)
tensor(1.4126e-17)
tensor(2.2629e-16)
tensor(2.2204e-16)
tensor(1.1840e-15)
tensor(8.9720e-17)
tensor(8.9301e-16)
tensor(1.4101e-16)
tensor(8.9251e-16)
tensor(1.8553e-15)
tensor(1.2527e-15)
tensor(2.2603e-16)
tensor(1.2998e-16)
tensor(9.3086e-16)
tensor(8.1040e-15)
tensor(3.5046e-16)
tensor(1.7764e-15)
tensor(6.4127e-18)
tensor(2.5272e-19)
tensor(1.1278e-15)
tensor(9.4464e-16)
tensor(8.8818e-16)
tensor(3.0251e-16)
tensor(1.8007e-15)
tensor(2.4058e-16)
tensor(2.2655e-16)
tensor(1.255

  7%|▋         | 588/7996 [00:00<00:03, 1912.56it/s]

tensor(1.7817e-15)
tensor(1.1252e-15)
tensor(8.9484e-16)
tensor(2.4223e-16)
tensor(1.1244e-15)
tensor(9.2640e-16)
tensor(8.9324e-16)
tensor(9.6643e-16)
tensor(6.1988e-17)
tensor(2.3404e-16)
tensor(9.1357e-16)
tensor(1.8319e-15)
tensor(1.7915e-15)
tensor(1.1253e-15)
tensor(8.9315e-16)
tensor(1.7825e-15)
tensor(9.7086e-16)
tensor(5.1205e-35)
tensor(8.9052e-16)
tensor(1.7818e-15)
tensor(3.9304e-18)
tensor(8.9469e-16)
tensor(3.6276e-15)
tensor(9.4369e-16)
tensor(9.4865e-16)
tensor(7.1838e-15)
tensor(1.1700e-15)
tensor(2.7768e-16)
tensor(1.8319e-15)
tensor(9.0593e-16)
tensor(1.1453e-15)
tensor(8.8926e-16)
tensor(9.1275e-16)
tensor(1.8319e-15)
tensor(1.7958e-15)
tensor(5.5199e-18)
tensor(2.2644e-16)
tensor(3.4694e-18)
tensor(2.9016e-15)
tensor(3.4694e-18)
tensor(1.7843e-17)
tensor(9.4375e-16)
tensor(8.9539e-16)
tensor(1.3878e-17)
tensor(2.2424e-16)
tensor(9.5216e-16)
tensor(5.8279e-17)
tensor(9.5371e-16)
tensor(9.0324e-16)
tensor(9.0898e-16)
tensor(5.4098e-18)
tensor(2.1053e-15)
tensor(4.231

 12%|█▏        | 987/7996 [00:00<00:03, 1960.43it/s]

...

tensor(1.9678e-17)
tensor(9.4369e-16)
tensor(4.4745e-15)
tensor(5.0510e-16)
tensor(1.7915e-15)
tensor(1.8380e-15)
tensor(2.3996e-16)
tensor(3.5066e-16)
tensor(2.6459e-16)
tensor(8.9461e-16)
tensor(9.5802e-16)
tensor(1.1346e-15)
tensor(5.6199e-17)
tensor(9.4369e-16)
tensor(1.8416e-15)
tensor(9.6563e-16)
tensor(2.0275e-16)
tensor(1.0511e-15)
tensor(3.8819e-15)
tensor(3.7955e-15)
tensor(4.5403e-15)
tensor(1.5892e-17)
tensor(5.5511e-17)
tensor(9.1256e-16)
tensor(9.0269e-15)
tensor(2.5000e-16)
tensor(2.2204e-16)
tensor(2.3113e-16)
tensor(7.6769e-17)
tensor(5.9259e-17)
tensor(1.1186e-15)
tensor(1.1465e-15)
tensor(9.4903e-16)
tensor(4.4451e-15)
tensor(5.5511e-17)
tensor(1.0556e-15)
tensor(1.1112e-15)
tensor(2.4980e-15)
tensor(1.1102e-16)
tensor(1.1743e-15)
tensor(9.8348e-18)
tensor(1.5039e-17)
tensor(2.0721e-15)
tensor(2.9507e-15)
tensor(8.8917e-16)
tensor(9.5006e-16)
tensor(9.4903e-16)
tensor(8.8818e-16)
tensor(9.2728e-16)
tensor(1.1785e-15)
tensor(2.5493e-16)
tensor(2.3821e-16)
tensor(8.917

 96%|█████████▋| 7699/7996 [00:03<00:00, 1959.19it/s]

tensor(3.7841e-17)
tensor(1.1317e-15)
tensor(1.0351e-15)
tensor(2.9520e-15)
tensor(1.8515e-15)
tensor(6.3765e-18)
tensor(9.0934e-16)
tensor(2.4671e-16)
tensor(9.5576e-16)
tensor(1.1263e-15)
tensor(8.9305e-16)
tensor(1.6180e-17)
tensor(2.0184e-15)
tensor(3.8542e-15)
tensor(3.5565e-15)
tensor(1.0131e-15)
tensor(2.3408e-16)
tensor(3.6426e-15)
tensor(6.5135e-15)
tensor(5.1024e-17)
tensor(6.1137e-17)
tensor(1.0593e-15)
tensor(1.8008e-15)
tensor(9.4971e-16)
tensor(6.8353e-17)
tensor(4.8148e-35)
tensor(9.5718e-16)
tensor(2.9093e-15)
tensor(3.5540e-15)
tensor(1.1176e-15)
tensor(9.0436e-16)
tensor(3.6524e-15)
tensor(1.8368e-15)
tensor(2.9439e-17)
tensor(6.5050e-15)
tensor(9.1793e-16)
tensor(9.2052e-15)
tensor(1.8166e-15)
tensor(2.3617e-16)
tensor(2.5691e-16)
tensor(2.3617e-16)
tensor(3.6334e-15)
tensor(1.1546e-15)
tensor(2.0727e-15)
tensor(3.7817e-15)
tensor(1.3878e-17)
tensor(8.8818e-16)
tensor(3.6545e-15)
tensor(1.2417e-15)
tensor(6.0881e-17)
tensor(4.4964e-15)
tensor(5.2401e-15)
tensor(1.782

100%|██████████| 7996/7996 [00:04<00:00, 1952.48it/s]

tensor(1.8284e-15)
tensor(2.2574e-16)
tensor(1.3295e-15)
tensor(1.8319e-15)
tensor(1.7775e-15)
tensor(1.8319e-15)
tensor(1.7853e-15)
tensor(1.9509e-15)
tensor(9.0206e-16)
tensor(3.1570e-16)
tensor(2.0027e-15)
tensor(1.1175e-15)
tensor(3.4750e-16)
tensor(1.8552e-15)
tensor(9.0497e-16)
tensor(2.5452e-16)
tensor(1.4436e-15)
tensor(2.0268e-15)
tensor(2.3402e-16)
tensor(9.3831e-16)
tensor(2.2646e-16)
tensor(1.8008e-15)
tensor(4.5937e-16)
tensor(3.7472e-17)
tensor(1.7817e-15)
tensor(3.5677e-16)
tensor(2.2425e-16)
tensor(9.0313e-16)
tensor(2.2229e-16)
tensor(3.5718e-15)
tensor(9.3261e-16)
tensor(1.8319e-15)
tensor(8.8818e-16)
tensor(1.6180e-15)
tensor(9.5869e-16)
tensor(1.8134e-15)
tensor(6.3131e-18)
tensor(9.1262e-16)
tensor(7.3830e-16)
tensor(5.4215e-16)
tensor(1.0681e-16)
tensor(9.0724e-16)
tensor(6.3606e-17)
tensor(8.9717e-16)
tensor(5.1828e-16)
tensor(1.1101e-16)
tensor(1.1839e-15)
tensor(2.0248e-15)
tensor(3.8112e-15)
tensor(8.8477e-17)
tensor(2.2351e-16)
tensor(5.0939e-16)
tensor(1.228

In [19]:
sum(RQ_loss)/len(RQ_loss)

tensor(1.1316e-15)

In [16]:
df

,filename,line_file_name,rel_rot,rel_tran
0,rgb/2t7WUuJeko7/0_0_40.png,rgb/2t7WUuJeko7/0_0_40_line.csv,"[0.9207172033825632, 0.0, -0.38306085185081906...","[-0.11032094395045366, -0.5381260901192229, 2...."
1,rgb/2t7WUuJeko7/0_0_40.png,rgb/2t7WUuJeko7/0_0_40_line.csv,"[-0.22748901285929143, -6.938893903907228e-18,...","[-0.16339169781799, 0.5172137279301707, -2.507..."
2,rgb/2t7WUuJeko7/0_0_43.png,rgb/2t7WUuJeko7/0_0_43_line.csv,"[0.9092381248752545, -1.3660947373317356e-17, ...","[-0.44376420417281676, 0.3101704954060034, -1...."
3,rgb/2t7WUuJeko7/0_0_50.png,rgb/2t7WUuJeko7/0_0_50_line.csv,"[-0.8857617092010768, 4.336808689942018e-19, -...","[1.8277234233235014, 0.13601564158756485, -0.8..."
4,rgb/2t7WUuJeko7/0_0_16.png,rgb/2t7WUuJeko7/0_0_16_line.csv,"[0.9420741893214133, 0.0, -0.32924223785875323...","[-2.9098352717327325, 0.6938825382726449, -3.8..."
...,...,...,...,...
7991,rgb/Vt2qJdWjCF2/0_3_6.png,rgb/Vt2qJdWjCF2/0_3_6_line.csv,"[0.5063891871560897, 6.938893903907228e-18, 0....","[0.6227922010345895, 0.4808561805800877, -2.18..."
7992,rgb/Vt2qJdWjCF2/0_3_25.png,rgb/Vt2qJdWjCF2/0_3_25_line.csv,"[0.44572446792149134, 0.0, -0.8787234085626714...","[-0.11075968388528268, 0.5689329565841225, -2...."
7993,rgb/Vt2qJdWjCF2/0_3_6.png,rgb/Vt2qJdWjCF2/0_3_6_line.csv,"[0.9991484224707612, 1.474514954580286e-17, -0...","[-1.6754211014830895, -0.17440628407532743, 0...."
7994,rgb/Vt2qJdWjCF2/0_3_8.png,rgb/Vt2qJdWjCF2/0_3_8_line.csv,"[0.3882533459712649, 0.0, 0.9046211602433875, ...","[1.2216756881081692, 0.6097304769122762, -3.32..."


In [ ]:
df = pd.concat([df1,df2])

In [ ]:
df1

In [17]:
df = df.reset_index(drop=True)

In [18]:
df

,filename,line_file_name,rel_rot,rel_tran
0,rgb/2t7WUuJeko7/0_0_40.png,rgb/2t7WUuJeko7/0_0_40_line.csv,"[0.9207172033825632, 0.0, -0.38306085185081906...","[-0.11032094395045366, -0.5381260901192229, 2...."
1,rgb/2t7WUuJeko7/0_0_40.png,rgb/2t7WUuJeko7/0_0_40_line.csv,"[-0.22748901285929143, -6.938893903907228e-18,...","[-0.16339169781799, 0.5172137279301707, -2.507..."
2,rgb/2t7WUuJeko7/0_0_43.png,rgb/2t7WUuJeko7/0_0_43_line.csv,"[0.9092381248752545, -1.3660947373317356e-17, ...","[-0.44376420417281676, 0.3101704954060034, -1...."
3,rgb/2t7WUuJeko7/0_0_50.png,rgb/2t7WUuJeko7/0_0_50_line.csv,"[-0.8857617092010768, 4.336808689942018e-19, -...","[1.8277234233235014, 0.13601564158756485, -0.8..."
4,rgb/2t7WUuJeko7/0_0_16.png,rgb/2t7WUuJeko7/0_0_16_line.csv,"[0.9420741893214133, 0.0, -0.32924223785875323...","[-2.9098352717327325, 0.6938825382726449, -3.8..."
...,...,...,...,...
7991,rgb/Vt2qJdWjCF2/0_3_6.png,rgb/Vt2qJdWjCF2/0_3_6_line.csv,"[0.5063891871560897, 6.938893903907228e-18, 0....","[0.6227922010345895, 0.4808561805800877, -2.18..."
7992,rgb/Vt2qJdWjCF2/0_3_25.png,rgb/Vt2qJdWjCF2/0_3_25_line.csv,"[0.44572446792149134, 0.0, -0.8787234085626714...","[-0.11075968388528268, 0.5689329565841225, -2...."
7993,rgb/Vt2qJdWjCF2/0_3_6.png,rgb/Vt2qJdWjCF2/0_3_6_line.csv,"[0.9991484224707612, 1.474514954580286e-17, -0...","[-1.6754211014830895, -0.17440628407532743, 0...."
7994,rgb/Vt2qJdWjCF2/0_3_8.png,rgb/Vt2qJdWjCF2/0_3_8_line.csv,"[0.3882533459712649, 0.0, 0.9046211602433875, ...","[1.2216756881081692, 0.6097304769122762, -3.32..."


In [19]:
df = df.drop_duplicates(['filename'])

In [20]:
df = df.reset_index(drop=True)

In [21]:
df

,filename,line_file_name,rel_rot,rel_tran
0,rgb/2t7WUuJeko7/0_0_40.png,rgb/2t7WUuJeko7/0_0_40_line.csv,"[0.9207172033825632, 0.0, -0.38306085185081906...","[-0.11032094395045366, -0.5381260901192229, 2...."
1,rgb/2t7WUuJeko7/0_0_43.png,rgb/2t7WUuJeko7/0_0_43_line.csv,"[0.9092381248752545, -1.3660947373317356e-17, ...","[-0.44376420417281676, 0.3101704954060034, -1...."
2,rgb/2t7WUuJeko7/0_0_50.png,rgb/2t7WUuJeko7/0_0_50_line.csv,"[-0.8857617092010768, 4.336808689942018e-19, -...","[1.8277234233235014, 0.13601564158756485, -0.8..."
3,rgb/2t7WUuJeko7/0_0_16.png,rgb/2t7WUuJeko7/0_0_16_line.csv,"[0.9420741893214133, 0.0, -0.32924223785875323...","[-2.9098352717327325, 0.6938825382726449, -3.8..."
4,rgb/2t7WUuJeko7/0_0_42.png,rgb/2t7WUuJeko7/0_0_42_line.csv,"[-0.5753437906912208, 0.0, -0.802884342487435,...","[4.068924753198918, 0.5133920823133029, -2.363..."
...,...,...,...,...
3076,rgb/Vt2qJdWjCF2/0_3_14.png,rgb/Vt2qJdWjCF2/0_3_14_line.csv,"[0.999870509937996, -1.3877787807814457e-17, -...","[0.729234787023369, -0.11964623656399787, 0.37..."
3077,rgb/Vt2qJdWjCF2/0_3_9.png,rgb/Vt2qJdWjCF2/0_3_9_line.csv,"[0.5274761051923089, -6.938893903907228e-18, 0...","[0.296733117502176, 0.31260874796247773, -1.30..."
3078,rgb/Vt2qJdWjCF2/0_3_13.png,rgb/Vt2qJdWjCF2/0_3_13_line.csv,"[0.6858069009503456, 1.3877787807814457e-17, -...","[-1.8384447714477585, -0.09597454249024429, 0...."
3079,rgb/Vt2qJdWjCF2/0_3_19.png,rgb/Vt2qJdWjCF2/0_3_19_line.csv,"[0.8809583340747859, 0.0, -0.46449993992119976...","[0.21062757801488985, 0.05708778215546917, -0...."


In [ ]:
df.to_csv('matterport_test_20230618.csv',index=False)

In [ ]:
import numpy as np

In [ ]:
k = np.array([3.14,2.9])
t = np.array([-1.7,2.6])

In [ ]:
np.cross(k,t)

In [ ]:
np.dot(k,t)